## 单机编程框架

### 数据流图
- 创建多个数据流图实例

In [ ]:
g1 = tf.Graph()
with g1.as_default():
    a = tf.Variable(0, name='a')
    assert a.graph is g1
with tf.Graph().as_default() as g2:
    b = tf.Variable(1, name='b')

### 单机编程框架

##### 这里添加了flags.DEFINE_string("f", '', "kernel")，不然会出现UnrecognizedFlagError: Unknown command line flag 'f'

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#超参
flags = tf.app.flags
flags.DEFINE_string("f", '', "kernel")
flags.DEFINE_string("data_dir", "data/", "Directionary for storing mnist data")
flags.DEFINE_float("learning_rate", 0.5, "Learning rate")
FLAGS = flags.FLAGS

Instructions for updating:
Use the retry module or similar alternatives.


In [30]:
def model():
    X = tf.placeholder(shape=[None, 784], dtype=tf.float32, name="X")
    Y = tf.placeholder(shape=[None, 10], dtype=tf.float32, name="Y")
    with tf.variable_scope('layer1', reuse=tf.AUTO_REUSE):
        W = tf.get_variable(name='W', shape=[784, 10], initializer=tf.random_normal_initializer())
        b = tf.get_variable(name='b', shape=[10], initializer=tf.constant_initializer(0.))
    y = tf.add(tf.matmul(X, W), b)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=Y))
    optimzer = tf.train.GradientDescentOptimizer(FLAGS.learning_rate)
    train_op = optimzer.minimize(cost)
    
    mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
#         sess.run(tf.global_variables_initializer())
        saver.restore(sess, 'args_save/')
        for step in range(1000):
            batch_x, batch_y = mnist.train.next_batch(100)
            sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
            if step%100 == 0:
                saver.save(sess, 'args_save/')
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print("acc=%s"%sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

In [29]:
#初始化训练
model()

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
acc=0.8768


In [31]:
#模型读取,继续训练
model()

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from args_save/
acc=0.8909


    模型中有很多名字很长的方法，变量,类似的命名，所以容易出错，可以利用自动补齐，不过还是要清楚命名以及命名规则，注意规范以及注释

### 单机会话和分布式会话的父类

In [ ]:
from tensorflow.python import pywrap_tensorflow as tf_session

class BaseSession(SessionInterface):
    
    def __init__(self, target='', graph=None, config=None):
        #省略数据流图加载和会话配置
        self._session = None
        opt =tf_session.TF_NewSessionOptions(target=self._target, config)
        try:
            with errors.raise_exception_on_not_ok_status() as status:
                self._session = tf_session.TF_NewSession(opts, status)
        finally:
            tf_session.TF_DeleteSessionOptions(opts)

    会话是通过调用C API定义的NewSession 方法实现的，传入target作为单机（空字符串或者None），分布式（...）参数

#### 关于运算节点的运算方式以及CPU，GPU管理参见P117，更多参考文档

## 分布式编程框架